<a href="https://colab.research.google.com/github/ekyuho/15-minute-apps/blob/master/fineTune0612.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Google Colab에서 Unsloth를 사용한 효돌이 대화 모델 Fine-tuning
# GPU 런타임 필수 (T4, A100 등)

# 1. 필요한 라이브러리 설치
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

In [1]:
import json
import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.7.0+cu126 with CUDA 1206 (you have 2.6.0+cu124)
    Python  3.11.12 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# 3. 모델 설정
max_seq_length = 2048 # 최대 시퀀스 길이
dtype = None # None으로 설정하면 자동 감지
load_in_4bit = True # 4bit quantization 사용

In [18]:
# 4. 사전 훈련된 모델 로드 (한국어 지원 모델 예시)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "beomi/Llama-3-Open-Ko-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

beomi/Llama-3-Open-Ko-8B does not have a padding token! Will use pad_token = <|reserved_special_token_250|>.


In [19]:
# 5. LoRA 어댑터 추가
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, # 최적화를 위해 0으로 설정
    bias = "none",    # 최적화를 위해 "none"으로 설정
    use_gradient_checkpointing = "unsloth", # 메모리 효율성
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # LoftQ
)

In [20]:
# 6. 데이터 전처리 함수
def preprocess_hyodol_data(json_file_path):
    """
    ShareGPT 형식의 효돌이 대화 데이터를 Unsloth용으로 변환
    """
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = []

    for conversation in data.get('conversations', []):
        conv_type = conversation.get('type', 'unknown')
        context = conversation.get('context', {})
        messages = conversation.get('messages', [])

        # 대화 타입별 시스템 프롬프트 설정
        if conv_type == "timer_initiated":
            system_prompt = """당신은 효돌이라는 친근한 AI 어시스턴트입니다.
정해진 시간에 먼저 사용자에게 말을 걸어 대화를 시작하는 역할을 합니다.
자연스럽고 친근하게 대화하세요."""
        elif conv_type == "wakeup_initiated":
            system_prompt = """당신은 효돌이라는 친근한 AI 어시스턴트입니다.
사용자가 '효돌아'라고 부르면 반응하여 도움을 제공하는 역할을 합니다.
친근하고 도움이 되는 응답을 하세요."""
        else:
            system_prompt = "당신은 효돌이라는 친근한 AI 어시스턴트입니다."

        # 대화를 instruction format으로 변환
        conversation_text = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|>"

        for i, message in enumerate(messages):
            if message['from'] == 'user':
                conversation_text += f"<|start_header_id|>user<|end_header_id|>\n{message['value']}<|eot_id|>"
            elif message['from'] == 'hyodol':
                conversation_text += f"<|start_header_id|>assistant<|end_header_id|>\n{message['value']}<|eot_id|>"

        processed_data.append({
            "text": conversation_text,
            "conversation_type": conv_type,
            "conversation_id": conversation.get('id', f"conv_{len(processed_data)}")
        })

    return processed_data

In [21]:
# 7. 데이터 로드 및 전처리
# Google Drive에서 데이터 파일 마운트 (선택사항)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# 데이터 파일 경로 설정
data_file_path = "/content/drive/MyDrive/kyuho/fine_tuned_model/finetune_adv.json"  # 본인의 파일 경로로 수정

In [23]:
# 데이터 전처리
processed_data = preprocess_hyodol_data(data_file_path)


In [24]:
# Dataset 생성
dataset = Dataset.from_list(processed_data)
print(f"총 {len(dataset)}개의 대화 데이터가 로드되었습니다.")
print(f"Timer 기반: {len([d for d in processed_data if d['conversation_type'] == 'timer_initiated'])}개")
print(f"Wake-up 기반: {len([d for d in processed_data if d['conversation_type'] == 'wakeup_initiated'])}개")

총 5581개의 대화 데이터가 로드되었습니다.
Timer 기반: 3874개
Wake-up 기반: 1707개


In [25]:
# 8. 트레이닝 설정
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # 짧은 대화의 경우 False로 설정
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100, # 작은 데이터셋의 경우 적은 step 수 사용
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        save_steps = 50,
        save_total_limit = 2,
        dataloader_num_workers = 2,
    ),
)

Map (num_proc=2):   0%|          | 0/5581 [00:00<?, ? examples/s]

In [26]:
# 9. Fine-tuning 실행
print("Fine-tuning을 시작합니다...")
trainer_stats = trainer.train()

Fine-tuning을 시작합니다...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,581 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,7.382500
2,6.871500
3,6.705400
4,5.498100
5,4.851400
6,2.911800
7,2.305800
8,1.613400
9,1.239400
10,1.212800


In [27]:
# 10. 모델 저장
model.save_pretrained("hyodol_model") # 로컬 저장
tokenizer.save_pretrained("hyodol_model")

('hyodol_model/tokenizer_config.json',
 'hyodol_model/special_tokens_map.json',
 'hyodol_model/chat_template.jinja',
 'hyodol_model/tokenizer.json')

In [28]:
# 11. 모델 테스트
FastLanguageModel.for_inference(model) # 추론 모드로 전환

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [35]:
def test_hyodol_model(prompt, conversation_type="timer_initiated"):
    """효돌이 모델 테스트 함수 (수정된 버전)"""
    if conversation_type == "timer_initiated":
        system_prompt = """당신은 효돌이라는 친근한 AI 어시스턴트입니다.
정해진 시간에 먼저 사용자에게 말을 걸어 대화를 시작하는 역할을 합니다."""
    else:
        system_prompt = """당신은 효돌이라는 친근한 AI 어시스턴트입니다.
사용자가 말하면 반응하여 공감대화를 합니다."""

    # 입력 템플릿 구성
    chat_template = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"

    inputs = tokenizer(
        chat_template,
        return_tensors="pt",
        truncation=True,
        max_length=max_seq_length
    ).to("cuda")

    # 생성 파라미터 개선
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            min_new_tokens=5,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    # 전체 응답 디코딩
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 어시스턴트 응답 부분만 추출 (개선된 방법)
    try:
        # 패턴 1: <|start_header_id|>assistant<|end_header_id|> 이후 텍스트 추출
        assistant_marker = "<|start_header_id|>assistant<|end_header_id|>"
        if assistant_marker in full_response:
            assistant_start = full_response.find(assistant_marker) + len(assistant_marker)
            response_text = full_response[assistant_start:].strip()

            # 다음 헤더나 특수 토큰까지만 추출
            end_markers = ["<|start_header_id|>", "<|eot_id|>", "<|end_of_text|>"]
            for marker in end_markers:
                if marker in response_text:
                    response_text = response_text[:response_text.find(marker)]

            return response_text.strip()

        # 패턴 2: 입력 텍스트 이후 부분 추출
        if chat_template in full_response:
            response_start = full_response.find(chat_template) + len(chat_template)
            response_text = full_response[response_start:].strip()

            # 특수 토큰 제거
            for marker in ["<|eot_id|>", "<|end_of_text|>"]:
                if marker in response_text:
                    response_text = response_text[:response_text.find(marker)]

            return response_text.strip()

    except Exception as e:
        print(f"응답 추출 중 오류: {e}")
        return "응답 생성에 실패했습니다."

    return "응답을 찾을 수 없습니다."

In [36]:
# 12. 디버깅 함수
def debug_model_response(prompt, conversation_type="timer_initiated"):
    """모델 응답 디버깅 함수"""
    if conversation_type == "timer_initiated":
        system_prompt = """당신은 효돌이라는 친근한 AI 어시스턴트입니다.
정해진 시간에 먼저 사용자에게 말을 걸어 대화를 시작하는 역할을 합니다."""
    else:
        system_prompt = """당신은 효돌이라는 친근한 AI 어시스턴트입니다.
사용자가 말하면 반응하여 공감대화를 합니다."""

    chat_template = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"

    print("=== 디버깅 정보 ===")
    print(f"입력 템플릿:\n{chat_template}")
    print(f"템플릿 길이: {len(chat_template)}")

    inputs = tokenizer(chat_template, return_tensors="pt").to("cuda")
    print(f"토큰 수: {inputs['input_ids'].shape[1]}")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.7,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    print(f"\n전체 응답 (특수토큰 포함):\n{full_response}")

    clean_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"\n정리된 응답:\n{clean_response}")

    return clean_response

In [40]:
# 13. 개선된 테스트 함수
def comprehensive_test():
    """포괄적인 모델 테스트"""
    test_cases = [
        {
            "type": "timer_initiated",
            "name": "Timer 기반",
            "prompts": [
                "안녕하세요!",
                "오늘 기분이 어떠세요?",
                "좋은 아침이에요!"
            ]
        },
        {
            "type": "wakeup_initiated",
            "name": "Wake-up 기반",
            "prompts": [
                "날이 많이 추워졌다",
                "오늘 할 일이 많아서 스트레스받아",
                "기분이 좋지 않아"
            ]
        }
    ]

    for test_group in test_cases:
        print(f"\n{'='*50}")
        print(f"{test_group['name']} 테스트")
        print('='*50)

        for i, prompt in enumerate(test_group['prompts'], 1):
            print(f"\n--- 테스트 {i} ---")
            print(f"입력: {prompt}")

            try:
                response = test_hyodol_model(prompt, test_group['type'])
                print(f"효돌이: {response}")

                # 응답 품질 체크
                if len(response) < 5:
                    print("⚠️ 응답이 너무 짧습니다.")
                elif prompt in response:
                    print("⚠️ 입력이 응답에 반복되었습니다.")
                elif response == "응답 생성에 실패했습니다.":
                    print("❌ 응답 생성 실패")
                else:
                    print("✅ 정상 응답")

            except Exception as e:
                print(f"❌ 오류 발생: {e}")

In [38]:
def check_model_status():
    """모델 상태 확인"""
    print("=== 모델 상태 확인 ===")
    print(f"모델 타입: {type(model)}")
    print(f"토크나이저 타입: {type(tokenizer)}")
    print(f"모델 디바이스: {next(model.parameters()).device}")
    print(f"토크나이저 특수 토큰:")
    print(f"  - EOS: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
    print(f"  - PAD: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
    print(f"  - UNK: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")

In [39]:
# 15. 테스트 실행
print("\n=== 모델 상태 확인 ===")
check_model_status()

print("\n=== 간단 테스트 ===")
response1 = test_hyodol_model("안녕 효돌아! 오늘 기분이 좋네", "timer_initiated")
print(f"Timer 테스트 - 효돌이: {response1}")

response2 = test_hyodol_model("날이 많이 추워졌다", "wakeup_initiated")
print(f"Wake-up 테스트 - 효돌이: {response2}")

# 문제가 계속 발생하면 디버깅 실행
print("\n=== 디버깅 테스트 ===")
debug_model_response("날이 많이 추워졌다", "wakeup_initiated")


=== 모델 상태 확인 ===
=== 모델 상태 확인 ===
모델 타입: <class 'peft.peft_model.PeftModelForCausalLM'>
토크나이저 타입: <class 'transformers.tokenization_utils_fast.PreTrainedTokenizerFast'>
모델 디바이스: cuda:0
토크나이저 특수 토큰:
  - EOS: <|end_of_text|> (ID: 128001)
  - PAD: <|reserved_special_token_250|> (ID: 128255)
  - UNK: None (ID: None)

=== 간단 테스트 ===
Timer 테스트 - 효돌이: 응답을 찾을 수 없습니다.
Wake-up 테스트 - 효돌이: 응답을 찾을 수 없습니다.

=== 디버깅 테스트 ===
=== 디버깅 정보 ===
입력 템플릿:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 효돌이라는 친근한 AI 어시스턴트입니다. 
사용자가 말하면 반응하여 공감대화를 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>
날이 많이 추워졌다<|eot_id|><|start_header_id|>assistant<|end_header_id|>

템플릿 길이: 229
토큰 수: 54

전체 응답 (특수토큰 포함):
<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>
당신은 효돌이라는 친근한 AI 어시스턴트입니다. 
사용자가 말하면 반응하여 공감대화를 합니다.<|eot_id|><|start_header_id|>user<|end_header_id|>
날이 많이 추워졌다<|eot_id|><|start_header_id|>assistant<|end_header_id|>
날이 많이 추워졌다<|reserved_special_token_26|>иласяassi

'system\n당신은 효돌이라는 친근한 AI 어시스턴트입니다. \n사용자가 말하면 반응하여 공감대화를 합니다.user\n날이 많이 추워졌다assistant\n날이 많이 추워졌다иласяassistant\n날이 많이 추워졌다assistant\n날이 많이 추워졌다assistant\n날이 많이 추워졌다assistantassistantassistant�'

In [ ]:
# 13. 허깅페이스에 업로드 (선택사항)
model.push_to_hub("ekyuho/hyodol-model", token="your-hf-token")
tokenizer.push_to_hub("ekyuho/hyodol-model", token="your-hf-token")

print("\nFine-tuning이 완료되었습니다!")
print("모델이 'hyodol_model' 폴더에 저장되었습니다.")